# Non-linear Decision Boundaries - Logistic Regression

Throughout this coding homework assignment, you will work with a well-known dataset in machine learning field: Wisconsin breast cancer dataset from UCI Machine Learning Repository.

First you will be guided to transit from linear regression on continuous observations to binary label variables. You will see why linear regression is not the best model to use when you try to predict a binary class label.

Second, you will review fundamental concepts of logistic regression you have seen in the course note and lecture. But the emphasis at this time is to visualize properties of sigmoid function, and you'll have a chance to implement loss function and decision rule etc of logistic regression you have seen in theory.

Third, you will visualize a simple logistic regression model using only one numerical feature with an intercept. You cannot find optimal weight vector at this point, but the pedagogical purpose of this part is to take advantage of all functions you implemented in the second part. So you can fully understand how and why logistic regression works, without being introduced to sklearn library.

Lastly, you will use all features provided in the breast cancer dataset, and build a logistic regression model using sklearn library. You will get hands on experience in data cleaning, EDA, model training, and evaluation metrics. This will give you enough exposure to functions of sklearn library related to binary classification problem.

In [ ]:
# import necessary library and setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Introduction: From Linear Regresson to Binary Classification

Recal a regression problem in machine learning is to construct a mapping function, which takes in argument as feature vector of a single data point in feature space, and ouputs a predicted value based on the input. The purpose of this type of problem is to approximate outputs of this function as close as possible to underlying true observed value in response variable. So far we have only seen linear models. A model is a linear model if it is a linear combination of features in the feature space. You should remember that there is always a closed form solution to the linear model given features and observations.

We talk about classification problem when you are introduced to support vector machine (SVM) in classification problem. Now in order to output a prediction of a binary class label, we cannot use traditional linear regression since it outputs continuous predictions. SVM produces a linear decision boundary in feature space, but it works well only when data is linearly separable.

Now, we are curious about the case where data is not linearly separable. Extending from linear models to non-linear decision functions might make our life easier. We further want to suggest a probablistic interpretation of classification problem. Given binary labels 0 and 1, we can simply interpret our prediction as a probability that a given data point takes label value of 1. 

First, we want to [load Wisconsin breast cancer dataset from sklearn library](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html). We start with a model with only one single feature "mean radius", which is the first column. You will observe how linear regression model performs by visualizing its decision boundary. With this intuition in mind, you will understand why we want to introduce logistic regression today.

In [ ]:
# import breast cancer dataset from sklearn library
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
X_simple = pd.DataFrame(dataset.data, columns=dataset.feature_names)[['mean radius']]
y_simple = dataset.target

In [ ]:
# train test split
from sklearn.model_selection import train_test_split

X_simple_train, X_simple_test, y_simple_train, y_simple_test = train_test_split(X_simple, y_simple, test_size = 0.2)

In [ ]:
X_simple.head()

**Question: Visualize training set in a 2-D plane. Plot feature `mean radius` on x-axis and response variable `diagnosis` on y-axis.** 

In [ ]:
# visualize training set
plt.figure(figsize=(10, 6))

# TODO: use scatterplot to visualize training set

plt.ylabel('diagnosis')
plt.title('Mean radius vs Breast Cancer Diagnosis');

**Question: Comment on whether we are given a linearly separable data. Can you guess what type of model linear regression will output? What will be the decision boundary if we use support vector machine?**

**Question: Implement the following functions to solve for and visualize ordinay least square linear regression without regularization. `ordinary_least_square` finds close form solution to optimal weight vector in linear regression. `ols_predict` uses weight vector to predict a new test point. `mse` compute mean squared error of linear regression model on a given dataset.**

In [ ]:
# TODO: solve ordinary least square linear regression solution
def ordinary_least_square(X, y):
  return ...

# TODO: linear regression prediction
def ols_predict(X, w):
  return ...

# TODO: compute mean squared error
def mse(y, y_pred):
  return ...

Now you want to fit a linear regression model to this training set, only using "mean radius" feature and an extra intercept term. That is, our linear regression model should be in the form:

$$ \hat{y} = \hat{w}x + \hat{b} $$.

Use what you learned from previous week about linear regression to find close form solution to $w$ and $b$.

**Question: First append an intercept term to feature dataframe. Then use your defined function to solve for optimal least square solution**

In [ ]:
# TODO: append intercept column to dataframe
X_simple_train.loc[:, 'intercept'] = ...
X_simple_test.loc[:, 'intercept'] = ...

In [ ]:
# TODO: solve for optimal weight vector in simple linear regression model
# here we know optimal weight vector only has two terms
vec_ols = ...

**Question: Compute predictions of linear regression model on training set. Evaluate and report MSE on training set. Visualize predictions of linear regression model, and compare with true binary label values.**

In [ ]:
# TODO: predict label values on training set
y_pred_simple_train = ...

# TODO: append prediction column to X_simple_train
X_simple_train...

# TODO: compute MSE of linear regression model on training set
ols_simple_train_mse = ...

print("MSE of linear regression model with one feature and intercept term on training set:", ols_simple_train_mse)

In [ ]:
# visualize predictions of linear regression model on training set 
plt.figure(figsize=(10, 6))

# TODO: use scatterplot to visualize true label value of diagnosis, same as what you did in the previous part
#       use lineplot to visulize predicted value given by linear regression model

plt.title('Linear Regression Prediction on Training Set: One feature with an Intercept Term');

Use the generic rule, classify the data point as label 1 if ordinary least square regression returns a predicted value greater than or equal to 1/2, and label value 0 otherwise.

**Question: Add ordinary least square predicted labels as a column to `X_simple_train` as `ols_pred_label`. Report training accuracy of linear regression.**

In [ ]:
# TODO: Compute label value predicted by ols and add the corresponding column

In [ ]:
# TODO: Compute training accuracy of ordinary least square model

**Question: Perform the same procedure above on test set. What similarity do you observe? What do you want to say about linear regression model on classification problem?**

In [ ]:
# TODO: predict label values on test set
y_pred_simple_test = ...

# TODO: append prediction column to X_simple_train
X_simple_test...

# TODO: compute MSE of linear regression model on training set
ols_simple_test_mse = ...

print("MSE of linear regression model with one feature and intercept term on test set:", ols_simple_test_mse)

In [ ]:
# visualize predictions of linear regression model on test set 
plt.figure(figsize=(10, 6))

# TODO: use scatterplot to visualize true label value of diagnosis, same as what you did in the previous part
#       use lineplot to visulize predicted value given by linear regression model

plt.title('Linear Regression Prediction on Test Set: One feature with an Intercept Term');

## Theories of Logistic Regression: Fundamental Concepts

You shold observe from previous example, that linear regression model on classification problem doesn't perform well as expected. Linear regression model predicts a continuous numerical value from given feature space, but it's difficult for us to transform that numerical value into an 0-1 binary label. These values are not strictly between 0 and 1, so we cannot make this transformation without adding other heuristics. As you should see from previous weeks, ordinary least square regression model can be sensitive to outliers, and is thus unreliable.

This is where logistic regression models plays an important role in classification. You'll start examine binary classification problem in this assignment, and we'll introduce its extension to multiclass classification problem in the course note.

### Definition

In a binary classification problem, the logistic regression model predicts the **probability** that the **binary** response variable $Y$ takes the value of 1 given the features $x \in R^d$:

$$ P(Y = 1|x) = f_\theta(x) = \frac {1} {1 + \exp(-\sum_{k=1} ^{d} {\theta_k x_k})}$$

Similar to linear regression model, **parameters** we want to learn in logistic regression model is weight vector $\theta \in R^d$. You should observe that in prediction, we still have predicted probability as a **function** of $\theta^T x$. So logistic regression can usually be interpreted as an extention of linear model, called **generalized linear model**.

### Logistic Activation Function

Logistic regression model derives from **logisitic function**, called **sigmoid**:
$$ s(t) = \frac {1} {1 + e^{-t}} $$

You should observe that sigmoid function takes in a feature vector in $R^d$ and outputs a probability between 0 and 1. This is how logistic regression always work as intended in binary classification problem. Sigmoid function is so commonly used in ML field, as you can see more of it's applications in the following week when we talk about neural network.

**Question: Let's look at properties of sigmoid function. Implement sigmoid function in python.**

In [ ]:
# TODO: implement sigmoid function
def sigmoid(t):
  return ...



*   Domain of $s(t)$: $-\infty < t < \infty$
*   Range of $s(t)$: $0 < s(t) < 1$
*   Threshold Value: $s(0) = 0.5$
*   Reflection and Symmetry of $s(t)$: 
$$ 1 - s(t) = 1 - \frac {1} {1 + e^{-t}} = \frac {e^{-t}} {1 + e^{-t}} = s(-t) $$




In [ ]:
# plot shape of logistic function
t = np.linspace(-8, 8, 100)
plt.plot(t, sigmoid(t), lw=2) 
plt.xticks(range(-8, 9))
plt.yticks(np.arange(0, 1.1, 0.25))
plt.xlabel('$t$')
plt.ylabel('$s(t)$')
plt.title('logistic function s(t)');

In [ ]:
# plot symmetry of logistic function
t = np.linspace(-4, 4, 100)
plt.plot(t, sigmoid(t), lw=2) 
plt.xticks(range(-4, 5))
plt.yticks(np.arange(0, 1.1, 0.25))
plt.plot([1, 1], [sigmoid(1), 1], lw=2, color='red')
plt.plot([-1, -1], [0, sigmoid(-1)], lw=2, color='green')
plt.xlabel('$t$')
plt.ylabel('$s(t)$')
plt.title('logistic function s(t)');


*   Inverse of $s(t)$: 
$$ s(t) = z = \frac {1} {1 + e^{-t}}, 0 < z < 1 $$
$$ e^{-t} = \frac {1-z} {z}$$
$$ t = -\log(\frac {1-z} {z}) = \log(\frac {z} {1-z}), 0 < z < 1 $$
*   Derivative of $s(t)$ using chain rule:
$$ s'(t) = -\frac {-e^{t}} {(1 + e^{-t})^2} = s(t)(1 - s(t)) $$



### Decision Rule

Let's look at how logistic activation function exactly solves binary classification problem. Recall you saw in course note that output of sigmoid function is interpreted as probability that data point takes label 1. Given $s(0) = \frac {1}{2}$, intuitively we can use the sign of sigmoid function as decision rule to classify 0 or 1. This is a generalization to decision rule when we try to predict label of a test point, and this is indeed how `sklearn` implements logistic regression model:
*   **If $s(\hat {\theta}^Tx) \ge \frac{1}{2}$, predict this data point as class 1.**
*   **If $s(\hat {\theta}^Tx) < \frac{1}{2}$, predict this data point as class 0.**

You can think of this decision rule as using a **"threshold"** value of $\frac {1}{2}$ based on sigmoid function. Then this threshold value is a hyperparameter in logistic regression model. As always, you can use cross validation to tune this hyperparameter with custom values. It's not always true that $\frac{1}{2}$ is the best threshold to implement the decision rule on unseen test data.

**Question: Now implement decision function of our naive classifier. It takes input of a feature vector $x \in R^d$, a weight vector $\theta \in R^d$, and ouputs a prediction of binary label 0 or 1 for this data point.**

In [ ]:
# TODO: implement decision funcition of logistic regression model
def decision_rule(x, theta):
  return ...

### Loss Function

Recall in linear regression, we use mean squared error (MSE) as loss function. We find optimal weight vector by minimizing the squared loss on training set.

However, MSE doesn't work as intended in classification problem. In classification problem, each prediction happens in terms of the probability of a data point being label 1. For example, a test point has observed true label of 1, and our classifier predicts its probability of 1 as 0.6. Based on decision rule, the classifier will report a prediction of 1. MSE of this data point will report 0. However, we know the actual predicted probability is 0.6, still far away from true probability 1. So we are still far from minimizing the loss, and we need to define a new loss function that can account for this 0.4 gap.

This is how **negative log-likelihood function** as loss in classification comes from. It is also called **cross entropy** loss.

Given training points $(x_i, y_i), i = 1...n$, cross entropy loss is defined as

$$ L(\theta) = - \sum_{i=1}^{n} {y_i\log(f_\theta(x_i)) + (1-y_i)\log(1 - f_\theta(x_i))} $$

Unlike squared loss, cross entropy loss is not a convex function, so there's no close form solution to it. Usually you need to use gradient descent to find a global optimum with respect to data.

**Question: Implement cross entropy loss function below as `cross_entropy_loss`. It should take in arguments `labels`, which are true labels, and `pred_probs`, which are predicted probabilities, and ouput cross entropy loss of this classifier on this dataset.**

In [ ]:
# TODO: implement cross entropy loss function
def cross_entropy_loss(labels, pred_probs):
  return ...

## A Closer Look into Logistic Regression: One Feature with an Intercept Term

Now you will redo what you did in first part, predicting binary class label of Wisconsin Breast Cancer Dataset with only one feature "mean radius" and an extra intercept term. But now you want to choose a logistic regression model.

Recall that logistic regression model can be interpreted as a generalized linear regression model. Feature space is first linearly parametrized by a weight vector $t = wx + b $, and then processed by sigmoid function $s(t)$ that turns a numerical value to a 0-1 probability. Since logistic weights cannot be minimized in close form formula, you want to spend sometime to explore how to improve prediction accuracy manually.

**Question: You can start with an arbitrary weight vector, corresponding to one feature "mean radius" and an extra intercept term. Assign this value you choose to `vec_lr`, it should be 2 dimensional.** 

A good hint to start is notice visualization of training and test points has reverse shape of signmoid function we plotted in previous part. What does this imply about sign of $w$ in our logistic regression model?

In [ ]:
# TODO: select weight vector parameter
# you may want to change parameter values in vec_lr
vec_lr = np.array([..., ...]).reshape(2, 1)
vec_lr

In [ ]:
# visualize structure of `X_simple_train`
X_simple_train

**Question: Use the weight vector `vec_lr` you choose, and feed it into your logistic regression model. Use function you defined in the previous part, to compute predicted probability of each training sample has label value 1.**

In [ ]:
# TODO: predicted probability of logistic regression model on training set
# HINT: `y_pred_probs_simple_train` should be a column vector in same length as number of training samples
y_pred_probs_simple_train = ...

# append logistic regression predicted probabilities to X_simple_train
X_simple_train['lr_pred_probs'] = y_pred_probs_simple_train

In [ ]:
X_simple_train.head()

**Question: Compute MSE and cross entropy loss of logistic regression model on training set.**

In [ ]:
# TODO: compute mse
lr_simple_train_mse = ...
print("MSE of linear regression model with one feature and intercept term on training set:", lr_simple_train_mse)

# TODO: compute cross entropy loss
# Solution
lr_simple_train_entropy = cross_entropy_loss(y_simple_train, y_pred_probs_simple_train[0])
print("Cross entropy loss of linear regression model with one feature and intercept term on training set:", lr_simple_train_entropy)

**Question: Now we want to visualize decision boundary of logistic regression model. As before in linear regression model, use [scatterplot](https://seaborn.pydata.org/generated/seaborn.scatterplot.html) for true label and feature values, and use [lineplot](https://seaborn.pydata.org/generated/seaborn.lineplot.html#seaborn.lineplot) for predicted probabilities given by logistic regression model.**

In [ ]:
# visualize logistic regression predictions
plt.figure(figsize=(10, 6))

# TODO: use scatterplot to visualize true label value of diagnosis, same as what you did in the previous part
#       use lineplot to visulize predicted value given by linear regression model

plt.title('Logistic Regression Prediction on Training Set: One feature with an Intercept Term');

We want to use general decision rule: predict label 1 if sigmoid function gives probability is greater than 1/2, and label 0 otherwise.

**Question: add predicted label value as a column `lr_pred_label` to `X_simple_train` and report training accuracy.**

In [ ]:
# TODO: Add predicted label value column of logistic regression model to X_simple_train


In [ ]:
# TODO: report training accuracy of logistic regression model using general threshold of 1/2


Now from visualization, you can think of logistic regression model as approximating some transformation of sigmoid function as close to distribution of true observed labels in the training set. The way we use gradient descent to minimize cross entropy loss is to consistently changing shape of sigmoid function of logistic regression model, and find an optimal solution.

**Question: To complete fitting this simple logistic regression model, you want to evaluate its predicted probability on test set. Complete the same process as the above on test set, and report your observations.**

In [ ]:
# TODO: predicted probability of logistic regression model on test set
y_pred_probs_simple_test = ...

# append logistic regression predicted probabilities to X_simple_test
X_simple_test['lr_pred_probs'] = y_pred_probs_simple_test

In [ ]:
X_simple_test.head()

In [ ]:
# TODO: compute mse
lr_simple_test_mse = ...
print("MSE of linear regression model with one feature and intercept term on test set:", lr_simple_test_mse)

# TODO: compute cross entropy loss
lr_simple_test_entropy = ...
print("Cross entropy loss of linear regression model with one feature and intercept term on test set:", lr_simple_test_entropy)

In [ ]:
# visualize logistic regression predictions
plt.figure(figsize=(10, 6))

# TODO: use scatterplot to visualize true label value of diagnosis, same as what you did in the previous part
#       use lineplot to visulize predicted value given by linear regression model

plt.title('Logistic Regression Prediction on Test Set: One feature with an Intercept Term');

**Question: Compare your logistic regression model with one feature and an intercept term with linear regression model with same features. In what sense they are similar? In what sense they are different? What are your takeaways from current training and test accuracy? How do you want to further improve your logistic regression model on classification without adding more features?**

## Optimize your Simple Logistic Regression Model: One Feature with an Intercept Term

In this section, we'll perform two ways to find optimal parameter of logistic regression model: gradient descent and hyperparameter tuning with cross validation.

### Gradient Descent in Logistic Regression Model

Last week, you are introduced to gradient descent algorithm, and implement stochastic gradient descent on some dataset. Now you will apply the same thought process on logistic regression. The model parameter we want to optimize is `vec_lr` in $R^2$. The loss function we want to minimize is cross entropy loss as defined in the previous part. Here we will walk through and apply gradient descent algorithm on logistic regression model, and find the minimized loss.


These are common variations of gradient descent algorithms that we mentioned in the course note. $\alpha$ is step size. $s_i$ is predicted probability of data point $i$.

*   Batch gradient descent:
$$ \theta^{(t+1)} = \theta^{(t)} + \alpha \cdot \frac{1}{n} \sum_{i=1}^{n} { (y_i - s_i) X_i } $$
*   Stochastic gradient descent:
$$ \theta^{(t+1)} = \theta^{(t)} + \alpha \cdot (y_i - s_i)X_i $$
*   Mini-batch gradient descent:
$$ \theta^{(t+1)} = \theta^{(t)} + \alpha \cdot \frac{1}{|B|} \sum_{i \in B} {(y_i - s_i)X_i} $$





#### Batch Gradient Descent

**Question: Implement batch gradient descent on training set, with respect to cross entropy loss function. For simplicity, you will perform in total 1000 iterations, with step size 0.001 defined for you. You may want to first change pandas dataframe of `X_simple_train` to a numpy array to better work with matrix vector multiplication.**

In [ ]:
# due to time constraint, we will only implement 10000 iterations
num_iter = 10000

# set step size to be as small as 0.001
alpha = 0.1

# for simplicity, first transofmr training set features to numpy array
X_simple_train_grad = X_simple_train[['mean radius', 'intercept']].to_numpy()

In [ ]:
# TODO: Implement batch gradient descent on logistic regression model

# initialize with random value [0, 0]
vec_lr_batch = ...

# array to save cross entropy loss throughout process
loss_batch = ...

# implement batch gradient descent
for i in range(num_iter):

  # compute loss at current iteration
  ...

  # initiate gradient update vector
  ...

  # perform gradient update
  ...

**Question: Plot your cross entropy loss on training set with respect to number of iterations using batch gradient descent. You should see that the loss is consistently decreasing monotonely. Then report the final cross entropy loss value that batch gradient descent converges to.**

In [ ]:
# TODO: visulize change in cross entropy loss with respect to number of iterations

plt.figure(figsize=(12, 10))

x_axis = ...
...

plt.xlabel('number of iterations')
plt.ylabel('cross entropy loss')
plt.title('Cross Entropy Loss on Training Set vs Number of Iterations in Batch Gradient Descent');

In [ ]:
# TODO: Report final cross entropy loss on training set that batch gradient descent converges to

print('Final cross entropy loss on training set with batch gradient descent:', ...)

**Question: Use the optimal model parameter found by batch gradient descent to compute predicted probability given by logistic regression model. Visualize the decision boundary, i.e. shape of sigmoid function, compared to true label value.**

In [ ]:
# final logistic regression model parameter by batch gradient descent
vec_lr_batch

In [ ]:
# TODO: predicted probability of logistic regression model on training set
# HINT: `y_pred_probs_simple_train` should be a column vector in same length as number of training samples

...
batch_probs = ...

In [ ]:
# visualize logistic regression predictions
plt.figure(figsize=(10, 6))

# TODO: use scatterplot to visualize true label value of diagnosis, same as what you did in the previous part
#       use lineplot to visulize predicted value given by batch logistic regression model

...
...
plt.title('Logistic Regression Prediction on Training Set: Batch Gradient Descent');

**Question: Now compute training and test accuracy of batch gradient descent solution to logistic regression model.**

In [ ]:
# TODO: compute predicted labels of batch logisitc regression model
y_pred_label_batch = ...

# TODO: Compute training accuracy:
print('Training accuracy of Batch Logistic Regression model:', 
      ...)

In [ ]:
# TODO: compute predicted labels of batch logisitc regression model
...
batch_test_probs = ...

y_pred_test_label_batch = ...

# TODO: Compute test accuracy:
print('Test accuracy of Batch Logistic Regression model:', 
      ...)

#### Stochastic Gradient Descent

**Question: Implement stochastic gradient descent on training set, with respect to cross entropy loss function. Use same set of parameters as in the previous part.**

In [ ]:
# TODO: Implement batch gradient descent on logistic regression model

# initialize with random value [0, 0]
vec_lr_stochastic = ...

# array to save cross entropy loss throughout process
loss_stochastic = ...

# implement batch gradient descent
for i in range(num_iter):

  # compute loss at current iteration
  ...

  # initiate gradient update vector
  ...
  
  # perform gradient update
  ...

In [ ]:
# TODO: visulize change in cross entropy loss with respect to number of iterations

plt.figure(figsize=(12, 10))

x_axis = ...
...

plt.xlabel('number of iterations')
plt.ylabel('cross entropy loss')
plt.title('Cross Entropy Loss on Training Set vs Number of Iterations in Stochastic Gradient Descent');

In [ ]:
# TODO: Report final cross entropy loss on training set that stochastic gradient descent converges to

print('Final cross entropy loss on training set with stochastic gradient descent:', ...)

In [ ]:
# final logistic regression model parameter by stochastic gradient descent
vec_lr_stochastic

In [ ]:
# TODO: predicted probability of logistic regression model on training set
# HINT: `y_pred_probs_simple_train` should be a column vector in same length as number of training samples

...
stochastic_probs = ...

In [ ]:
# visualize logistic regression predictions
plt.figure(figsize=(10, 6))

# TODO: use scatterplot to visualize true label value of diagnosis, same as what you did in the previous part
#       use lineplot to visulize predicted value given by batch logistic regression model
...
...
plt.title('Logistic Regression Prediction on Training Set: Batch Gradient Descent');

**Question: Now compute training and test accuracy of stochastic gradient descent solution to logistic regression model.**

In [ ]:
# TODO: compute predicted labels of stochastic logisitc regression model
y_pred_label_stochastic = ...

# TODO: Compute training accuracy:
print('Training accuracy of Stochastic Logistic Regression model:', 
      ...)

In [ ]:
# TODO: compute predicted labels of stochastic logisitc regression model
...
stochastic_test_probs = ...

y_pred_test_label_stochastic = ...

# TODO: Compute test accuracy:
print('Test accuracy of Stochastic Logistic Regression model:', 
      ...)

#### Mini-Batch Gradient Descent

**Question: Implement mini-batch gradient descent on training set, with respect to cross entropy loss function. Use same set of parameters as in the previous part. Use same set of parameter as train test split, we will sample 20% of training points as batch size to compute gradient update.**

In [ ]:
# hyperparameter batch size
B = int(0.2 * X_simple_train_grad.shape[0])
B

In [ ]:
# TODO: Implement batch gradient descent on logistic regression model

# initialize with random value [0, 0]
vec_lr_mini_batch = ...

# array to save cross entropy loss throughout process
loss_mini_batch = ...

# implement batch gradient descent
for i in range(num_iter):

  # compute loss at current iteration
  ...

  # initiate gradient update vector
  ...

  # initiate gradient update vector
  ...

  # sample a random subset of training points with batch size B
  ...
  
  # perform gradient update
  ...

In [ ]:
# TODO: visulize change in cross entropy loss with respect to number of iterations

plt.figure(figsize=(12, 10))

x_axis = ...
...

plt.xlabel('number of iterations')
plt.ylabel('cross entropy loss')
plt.title('Cross Entropy Loss on Training Set vs Number of Iterations in Mini-Batch Gradient Descent');

In [ ]:
# TODO: Report final cross entropy loss on training set that mini-batch gradient descent converges to

print('Final cross entropy loss on training set with mini-batch gradient descent:', ...)

**Question: Use the optimal model parameter found by mini batch gradient descent to compute predicted probability given by logistic regression model. Visualize the decision boundary, i.e. shape of sigmoid function, compared to true label value.**

In [ ]:
# final logistic regression model parameter by batch gradient descent
vec_lr_mini_batch

In [ ]:
# TODO: predicted probability of logistic regression model on training set
# HINT: `y_pred_probs_simple_train` should be a column vector in same length as number of training samples
...
mini_batch_probs = ...

In [ ]:
# visualize logistic regression predictions
plt.figure(figsize=(10, 6))

# TODO: use scatterplot to visualize true label value of diagnosis, same as what you did in the previous part
#       use lineplot to visulize predicted value given by batch logistic regression model
...
...
plt.title('Logistic Regression Prediction on Training Set: Mini-Batch Gradient Descent');

**Question: Use the optimal model parameter found by stochastic gradient descent to compute predicted probability given by logistic regression model. Visualize the decision boundary, i.e. shape of sigmoid function, compared to true label value.**

In [ ]:
# TODO: compute predicted labels of stochastic logisitc regression model
y_pred_label_mini_batch = ...

# TODO: Compute training accuracy:
print('Training accuracy of Mini-Batch Logistic Regression model:', 
      ...)

In [ ]:
# TODO: compute predicted labels of stochastic logisitc regression model
...
mini_batch_test_probs = ...

y_pred_test_label_mini_batch = ...

# TODO: Compute test accuracy:
print('Test accuracy of Mini-Batch Logistic Regression model:', 
      ...)

**Question: Compare your optimal logistic greression model given by 3 types of gradient descent, with initial one you choose without optimize parameters. Comment on your observations.**

### Hyperparameter Tuning: Threshold Value in Prediction

We are interested in whether changing threshold value in prediction rule will improve our logistic regression model performance. For simplicity, we will use model parameter approximnately same as the result given by gradient descent algorithm.

In [ ]:
# Define model parameter for hyperparameter tuning.
vec_lr_tune = np.array([-1, 15]).reshape(2, 1)

**Question: Compute training accuracy of logistic regression model, with different threshold value between 0 and 1. To help you get started, recall currently we use threshold value of 0.5. Find out the best hyperparameter threshold value, using cross validation.**

In [ ]:
# TODO: train validation set split

X_tune_train, X_tune_val, y_tune_train, y_tune_val = ...

In [ ]:
# TODO: set candidate values for threshold
thresholds = ...

# TODO: Compute tune predictions on training and validation set
tune_train_scores = ...
tune_train_pred = ...
tune_val_scores = ...
tune_val_pred = ...

In [ ]:
# TODO: Compute training and validation accuracy of differnet threshold value

tune_train_accuracy = ...
tune_val_accuracy = ...

for ...:
  tune_train_pred_label = ...
  tune_val_pred_label = ...

  ...
  ...

**Question: Plot training and validation accuracy with respect to different threshold values.**

In [ ]:
# visualize hyperparameter tuning accuracy in training set
plt.figure(figsize=(10, 6))

# TODO: use lineplot to visualize training accuracy with respect to different threshold values
...

plt.xlabel('thresholds')
plt.ylabel('training accuracy')
plt.title('Logistic Rergession Model Accuracy on Training Set vs Different Threshold Values');

In [ ]:
# visualize hyperparameter tuning accuracy in training set
plt.figure(figsize=(10, 6))

# TODO: use lineplot to visualize validation accuracy with respect to different threshold values
...

plt.xlabel('thresholds')
plt.ylabel('validation accuracy')
plt.title('Logistic Rergession Model Accuracy on Validation Set vs Different Threshold Values');

**Question: Find best threshold value on training set and validation set. Are they the same? Which one you would choose to use in practice?**

In [ ]:
# TODO: Find best hyperparameter value that gives highest training accuracy
print('Best threshold value with highest training accuracy:', ...)

In [ ]:
# TODO: Find best hyperparameter value that gives highest validation accuracy
print('Best threshold value with highest validation accuracy:', ...)

## Compare Logistic Regression to Nearst Neighbor Perspective

As you visualize decision boundary of logistic regression model on 1 dimensional input, you might find it similar to something we discussed earlier this week.

We talk about using kernels earlier this week. Kernel function measures similarity between different data points in certain way. In classification problem, we are interested in how features of data points are similar under the condition that they have the same label value.

From kernel perspective, prediction on a new test point should be finding the closest data points in feature space, and taking certin weighted average of response values of these closest point. 

We now want to extend this idea to binary classification problem. Looking at the shape of logistic regression decision boundary above, we propose another way to generate such a similar decision boundary.

Given a new test point $x$, suppose we focus on $k$ closest point to $x$ in feature space. We predict the probability of $x$ has label $1$ in binary classification problem with the average of labe values of all $k$ closest point. This is reasonable intuitively, since data points closer to each other in feature space should have same label values. As you should have seen earlier this week, this is exactly how k-nearst neighbor algorithm works in regression and classification problem.

**Question: Now we want to implement k-nearst neighbor algorithm in sklearn, visualize its decision boundary, and compare its similarity to logistic regression decison boundary.**

In [ ]:
from sklearn.neighbors import NearestNeighbors

# TODO: change hyperparameter value k

k = ...
knn_model = NearestNeighbors(n_neighbors=k)
knn_model.fit(X_simple_train[['mean radius', 'intercept']])

In [ ]:
# TODO: implement knn predicted probabilities
# HINT: first you want to find k closest point from test point in training set
#       second you want to compute average of label values of these closest points
#       third predictions is given by this mean

knn_pred_probs = []
for i in range(X_simple_test.shape[0]):
  ...

In [ ]:
# visualize knn predictions
plt.figure(figsize=(10, 6))

# TODO: plot true label values and predicted values given by k nearst neighbor

plt.title('K Nearst Neighbor Prediction on Test Set: One feature with an Intercept Term');

**Question: Initialize with hyperparameter $k=5$. Plot your visualization of decision boundaries and report your observations. What does current decision boundary looks like? Do you feel it's a reasonable approach and can generate high test accuracy?**

**Question: Change hyperparameter value $k$, what do you observe when you increase value of $k$? Compare new decision boundary of k-nearst neighbor with large $k$ to logistic regression decision boundary. In what sense they are similar and dissimilar?**

**Question: After this trial, how do you want to interpret logistic regression model and power of kernel functions? What would you comment on using kernel and k-nearst neighbor to approximate logistic regression?**

## Application of Logistic Regression: Binary Classification in Breast Cancer Dataset

In the following part of this coding assignment, you will be looking at breast cancer dataset from UCI machine learning library. This is a well-known dataset in ML field. It includes several information of patients with or without cancer, and we'll use these information to build a binary logistic classifier to predict whether the patient has cancer or not. More details are at UCI machine learning website. 

### Import packages

Before we get started, we'll need to import packages necessary to our assignment: 

*   [pandas](https://pandas.pydata.org/pandas-docs/stable/) - a package for performing data analysis and manipulation
*   [numpy](https://numpy.org/doc/stable/contents.html) - a package for scientific computing
*   [matplotlib](https://matplotlib.org/3.3.2/contents.html) - the standard Python plotting package
*   [seaborn](https://seaborn.pydata.org/) - a dataframe-centric visualization package that is built off of matplotlib

In [ ]:
# import necessary library and setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load Dataset and Preliminary Understanding

The Wisconsin breast and cancer dataset is fully understood and analyzed in the ML field. Skit-learn library has a built-int [load_breast_cancer](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) function that imports this dataset. We no longer need to manually read csv files.

In [ ]:
# import breast cancer dataset from sklearn library
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()

In [ ]:
# convert to pandas dataframe 
features = pd.DataFrame(dataset.data, columns=dataset.feature_names)
labels = dataset.target

In [ ]:
# number of features and data points
print(features.shape)
print(labels.shape)

In [ ]:
# binary labels
list(dataset.target_names)

The entire dataset has 569 samples, there are in total 30 features for each patient sample. The diagnosis of each patient: M = malignant and B = benign corresponds to 0 and 1 values. Sklearn already partially preprocesses the dataset for us, patient ID column that is irrelevant to classfication has been removed.

Now let's look at what are feature columns in our dataset.

In [ ]:
# peek into features
features.head()

In [ ]:
# column names
print("Columns:", list(features.columns))

You can observe that we will deal with different measurements of a single parameter. For example, you will consider "mean texture" and "worst texture" as two features. You might wonder whether this may lead to high correlation between features. Hopefully you can explore this when you evaluate your model.

For reference, the following are complete attribute information of ten original real valued features from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)):

> Ten real-valued features are computed for each cell nucleus:

*   radius (mean of distances from center to points on the perimeter)
*   texture (standard deviation of gray-scale values)
*   perimeter
*   area
*   smoothness (local variation in radius lengths)
*   compactness (perimeter^2 / area - 1.0)
*   concavity (severity of concave portions of the contour)
*   concave points (number of concave portions of the contour)
*   symmetry
*   fractal dimension ("coastline approximation" - 1)











Let's look at whether there is a class imbalance in the dataset. You will examine number of label 0 in the dataset versus number of label 1 in the dataset.

In [ ]:
print("Number of non-cancer samples:", sum(labels == 0), "out of", len(labels), "samples in total.")
print("Number of cancer samples:", sum(labels == 1), "out of", len(labels), "samples in total.")

sns.countplot(x = labels)
plt.xlabel('diagnosis')
plt.title('Number of Benign-0 vs Malignant-1');

Fortunately class imbalance is not a significant problem here, since each single class accounts for approximately one half of total number of samples.

### Data Cleaning

Given a dataset to build machine learning model, you need to first deal with missing values in the dataset, if any. Remember there are different ways appriximate missing values: completely delete them, use low rank approximation, or use other statistics such as average of same feature.

If you completely delete samples with missing values, this might lead to bias in your model and generalization issues as you might ignore a specific portion in the population. If you use other statistics to replace missing values, be careful with their properties: whether they are sensitive to outliers and what implications they bring etc.

You will look at whether we need to deal with missing values in our data set.

In [ ]:
for column in features.columns:
  print("Column", column, "has", sum(features[column].isnull()), "missing values.")

There's no missing values you need to consider in this case. Fortunely, binary labels "M = malignment" and "B = belignment" are already been one-hot-encoded as numerical labels 1 and 0 correspondingly. So this dataset can be directly used for classification.

**Question: Now, construct a pandas dataframe contains means and standard deviation of each column in `features` table. You want to explore whether values of features are in similar range.**

In [ ]:
# TODO: assign mean and sd of each column and make a dataframe containing all values

feature_mean_sd = ...
feature_mean_sd

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='feature', y='average', data=feature_mean_sd)
plt.xticks(rotation=45)
plt.title('Mean of Features in Wisconsin Breast Cancer Dataset');

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='feature', y='standard deviation', data=feature_mean_sd)
plt.xticks(rotation=45)
plt.title('Stadard Deviation of Features in Wisconsin Breast Cancer Dataset');

**Question: Report your observations ontraining set. What is the range of different features in the training set? What data cleaning technques you may want to use right now?**

**Question: Noramlize dataset `features` loaded from `sklearn` library directly. You can reassign new dataframe to the same variable name without creating a new one.**

In [ ]:
# TODO: normalize dataset `features`
# HINT: find mean, std of each column in dataset
#       compute standard unit
features = ...
features.head()

### Exploratory Data Analysis

You should observe original dataset has 10 features, we compute mean, standard deviation, and extreme value of each feature, so we will use 30 features in total to make prediction. You will visualize features from these three perspectives respectively.

In [ ]:
# split features to groups
features_mean = np.array(features.columns[:10])
features_error = np.array(features.columns[10:20])
features_worst = np.array(features.columns[20:31])

# add label column to features
features['diagnosis'] = labels

#### Data Visualization of Mean Features

We want to observe distribution of all mean feature values, seperated by breast cancer diagnosis labels. We want to learn whether certain feature values are correlated to diagnosis of breast cancer.

**Question: Visualize 3 set of features respectively. Specifically, for each set of features, draw a [boxplot](https://matplotlib.org/3.3.2/api/_as_gen/matplotlib.pyplot.boxplot.html) revealing distribution of feature values, separated by diagnosis label values $\{0, 1\}$. Then draw a [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) revealing correlation between different features. You should remember that highly correlated features doesn't help improve performance of model.**

**Then based on boxplot, briefly describe distribution of values of features in different groups in terms of different label values.**

In [ ]:
# boxplot of distribution of mean festures, separated by breast cancer binary labels
temp = pd.concat([features['diagnosis'], features[features_mean]], axis=1)
temp = pd.melt(temp, id_vars="diagnosis", var_name="features", value_name='value')
plt.figure(figsize=(12,10))

# TODO: boxplot

plt.xticks(rotation=45)
plt.title("Distribution of Mean Feature Values");

*Solution: You shold observe that among all mean feature values, except "mean fractal dimension" has similar distribution among "M" and "B" labels, all other mean feature distributions have visible differences.*

Then you want to look at whether there are high correlation between differnet features in the same group.

In [ ]:
# heatmap of correlation between mean features
plt.figure(figsize=(10,10))

# TODO: heatmap

plt.title('Correlation between Mean Feature Values');

#### Data Visualization of Error Features

We want to observe distribution of all error feature values, seperated by breast cancer diagnosis labels. We want to learn whether certain feature values are correlated to diagnosis of breast cancer.

In [ ]:
# boxplot of distribution of error festures, separated by breast cancer binary labels
temp = pd.concat([features['diagnosis'], features[features_error]], axis=1)
temp = pd.melt(temp, id_vars="diagnosis", var_name="features", value_name='value')
plt.figure(figsize=(12,10))

# TODO: boxplot

plt.xticks(rotation=45)
plt.title("Distribution of Error Feature Values");

*Obviously, distribution of error features has higher variance. There are more outliers in values of variance than means. Except for "texture error", all other error features have visible difference in distribution relative to "M" and "B" labels.*

In [ ]:
# heatmap of correlation between error features
plt.figure(figsize=(10,10))

# TODO: heatmap

plt.title('Correlation between Error Feature Values');

#### Data Visualization of Worst Features

We want to observe distribution of all error feature values, seperated by breast cancer diagnosis labels. We want to learn whether certain feature values are correlated to diagnosis of breast cancer.

In [ ]:
# boxplot of distribution of worst festures, separated by breast cancer binary labels
temp = pd.concat([features['diagnosis'], features[features_worst]], axis=1)
temp = pd.melt(temp, id_vars="diagnosis", var_name="features", value_name='value')
plt.figure(figsize=(12,10))

# TODO: boxplot

plt.xticks(rotation=45)
plt.title("Distribution of Worst Feature Values");

*Solution: Fortunately, all worst features have distinct distribution for samples in "M" and "B" labels. This concludes that they are all valid features to be used in prediction.*

In [ ]:
# heatmap of correlation between worst features
plt.figure(figsize=(10,10))

# TODO: heatmap

plt.title('Correlation between Worst Feature Values');

### Logistic Regression Model 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

**Question: Set up features and observed labels in training set. Recall we already normalize `features` matrix, so no need to redo standardization. Use train test split with ratio of 80% - 20%.**

In [ ]:
# set up X and y matrix
features = features.drop('diagnosis', axis=1)
X = features
y = labels

# TODO: train-test split
X_train, X_test, y_train, y_test = ...

**Question: Now fit your training data logistic regression model imported from sklearn library, you may find documentations and examples [in this link](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) helpful.**

In [ ]:
# TODO: fit logistic regression model
model = ...
...

You should be able to know that sklearn solves optimal logistic regression weights using gridient descent in loss function, which you cannot implement here. 

**Question: Now let's simply evaluate our model, using training and test accuracy. Feel free to use builtin functions in sklearn.**

In [ ]:
# TODO: training accuracy of our model
train_accuracy = ...
print("Training accuracy of logistic regression classifier:", train_accuracy)

In [ ]:
# TODO: test accuracy of our model
test_accuracy = ...
print("Test accuracy of logistic regression classifier:", test_accuracy)

It's obvious for you to conclude that our logistic regression classifier does a pretty good job on both training and test set, with accuracy over 95% in general. But remember we talked about there are different perspectives to evaluate a machine learning model in classification problem.

### Evaluation of Binary Classifier

#### Confusion Matrix

You should be familiari with the role of confusion matrix in classification problem. 

**Question: Use sklearn library to report confusion matrix of your logistic regression classfier and report your observations. A heatmap might be a good choice here. We already import [metrics](https://scikit-learn.org/0.16/modules/classes.html#module-sklearn.metrics) from sklearn for you.**

In [ ]:
from sklearn import metrics

# TODO: find test confusion matrix
test_predictions = ...
cm = ...
cm

In [ ]:
# visualization of confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(data=cm, annot=True)
plt.title("Confusion Matrix of Logistic Classifier on Breast Cancer Test set");

There's only 5 points misclassified among test set. Half of them coming from either 0 or 1 label. We can only say the classifier performs consistently among two types.

#### Precision, Recall, TPR, and FNR

**Question: Based on confusion matrix above, compute FP, TN, FN, TP, precision, recall, TPR, and FPR of our classifier.**



*Solution:*

*Based on confusion matrix, test predictions are breaked down into following categories:*

*   *False Positives (FP): 2*
*   *True Negatives (TN): 35*
*   *False Negatives (FN): 3*
*   *True Positives (TP): 74*

*Evaluate value of precision, recall, TPR, and FNR:*

*   *precision = $\frac {TP} {TP + FP} $ = $\frac {74} {74 + 2}$ = $0.97368421$*
*   *recall = $\frac {TP} {TP + FN} $ = $\frac {74} {74 + 3}$ = $0.96103896$*
*   *TPR = recall = $0.96103896$*
*   *FPR = $\frac {FP} {FP + TN} $ = $\frac {2} {2 + 35}$ = 0.054054*





**Question: Now can can plot precision vs. recall with respect to each threshold. This is a similar metrics to ROC curve that we talked about before. You may find `predict_proba()` function of logistic regression in sklearn useful.**

In [ ]:
# get precision, recall, threshold
from sklearn.metrics import precision_recall_curve

# TODO: get predicted probability of LR, get precision and recall
test_pred_probs = ...
test_precision, test_recall, thresholds = ...

In [ ]:
# plot precision-recall curve
plt.figure(figsize=(12,10))

# TODO: use lineplot for precision-recall curve
#       make sure to add xlabel and ylabel

plt.xlabel('Test Recall')
plt.ylabel('Test Precision')
plt.title('Precision-Recall Curve of Logistic Regression Classifier on Breast Cancer Test Set');

#### ROC Curve and AUC-ROC

ROC curve is an important metric to evaluate a classification model. Plot ROC curve of logistic regression model, and report whether it's good or not based on your observations.

**Question: Plot ROC curve of logistic regression model on test set.** 

To get started, you will need to use sklearn metrics library to evalute. Remember logistic regression classifier in sklearn has a function `predict_proba()` that might be useful. You need to use probabilistic interpretation of logistic classifier: logistic regression predicts probability of a test point to have label 1. so you only cares about predicted probability of label 1.

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
# TODO: get ROC and AUC-ROC probability and score
test_pred_probs = ...
test_auc = ...
print("AUC of ROC score in test set:", test_auc)

In [ ]:
# plot ROC curve
test_fpr, test_tpr, thresholds = roc_curve(y_test, test_pred_probs)
plt.figure(figsize=(12,10))

# TODO: use lineplot for ROC curve
#       make sure to add xlabel and ylabel

plt.title('ROC Curve of Logistic Regression Classifier on Breast Cancer Test Set');

**Question: Comment on similarity and difference between ROC curve and precision-recall curve of our classifier. What does this say about performance of our logistic regression binary classifier?**

## Stochastic Gradient Descent

Now you want to use gradient descent to iteratively approximate optimal $\hat{\theta}$ by minimizing cross entropy loss function. Sklearn provides SGDClassifier model that implements stochastic gradient descent on logistic regression model, by minimizing cross entropy loss function.

In [ ]:
from sklearn.linear_model import SGDClassifier

**Question: Fit training set into [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) model. You may want to use custom parameter when you initiate the model, so that it performs classification task instead of regression task.**

In [ ]:
# TODO: construct stochastic gradient descent learning algorithm
sgd_model = ...
...

**Question: Reprt training accuracy and test accuracy of logistic regression model given by stochastic gradient descent.**

In [ ]:
# TODO: training accuracy of sgd model
sgd_train_accuracy = ...
print("Training accuracy of stochastic gradient descent classifier:", sgd_train_accuracy)

In [ ]:
# TODO: test accuracy of sgd model
sgd_test_accuracy = ...
print("Test accuracy of stochastic gradient descent classifier:", sgd_test_accuracy)

**Question: Print and visualize confusion matrix of stochastic gradient descent model.**

In [ ]:
# TODO: print and visualize confusion matrix
sgd_test_predictions = ...
sgd_cm = ...
sgd_cm

In [ ]:
# visualization of confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(data=sgd_cm, annot=True)
plt.title("Confusion Matrix of Stochastic GD Logistic Classifier on Breast Cancer Test set");

**Question: Find precision and recall values at different thresholds by SGD on test set. Plot precision vs recall curve as you did for LR model.**

In [ ]:
# TODO: find test predicted probabilities (NOT LABEL VALUES)
#       find precision, recall at different thresholds
sgd_test_pred_probs = ...
sgd_test_precision, sgd_test_recall, thresholds = ...

In [ ]:
# plot precision-recall curve
plt.figure(figsize=(12,10))

# TODO: use lineplot for precision vs recall curve
#       make sure to add xlabel and ylabel

plt.title('Precision-Recall Curve of SGD Logistic Classifier on Breast Cancer Test Set');

In [ ]:
# TODO: get ROC and AUC-ROC probability and score
sgd_test_pred_probs = ...
sgd_test_auc = ...
print("AUC of ROC score in test set:", test_auc)

In [ ]:
# plot ROC curve

# TODO: get FPR, TPR at different thresholds
sgd_test_fpr, sgd_test_tpr, thresholds = ...

plt.figure(figsize=(12,10))

# TODO: use lineplot for ROC curve
#       make sure to add xlabel and ylabel

plt.title('ROC Curve of SGD Logistic Classifier on Breast Cancer Test Set');

**Question: Observe performance of logistic regression model trained by stochastic gradient descent. How is it different from sklearn logistic regression model? Does stochastic gradient descent algorithm give us higher accuracy? How about value in confusion matrix and AUC-ROC? How would you connect to properties ot SGD we discussed last week?**

**Bonus: Change step size $\eta$ in stochastic gradient descent model you fit. Observe change in convergence. What's your takeaway if you want to use SGD to solve for optimal parameter in LR model?**

Congratulations on finishing this assignment! Hope you get intuition of how does logistic regression model exactly solve binary classification problem. Hope you understand why we introduce it as the most fundamental model in clssification problem, and as a generalized linear model. You should get hands on experiences about how to use sklearn to fit a logistic regression model, and how to evaluate from different perspectives.